<a href="https://colab.research.google.com/github/h3it-dias/FastCamp_Agent_AI_CEIA/blob/main/Agent_ReAct_Pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação de um agente ReAct para encontrar a força Peso

## Instalação da biblioteca Groq

Para utilizar LLMs gratuitos de forma mais eficiente, utilizou-se o Groq para o controle da APIs

In [7]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.8 MB/s eta 0:00:00


## Configuração da API do Groq

In [18]:
import os
from groq import Groq

os.environ['GROQ_API_KEY'] = "sua_api_key_aqui"

client = Groq(api_key=os.environ.get('GROQ_API_KEY'))

## Criação da classe Agent
Nessa etapa temos a criação da nossa classe Agent que é responsável por instaciar o agent que iremos usar.

In [25]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages_register = []
    if self.system is not None:
      self.messages_register.append({"role":"system", "content": self.system})

  def __call__(self, message=""):
    if message != "":
      self.messages_register.append({"role":"user", "content": message})
    result = self.execute()
    self.messages_register.append({"role":"assistant", "content": result})
    return result

  def execute(self):
    completion_model = client.chat.completions.create(
      messages=self.messages_register,
        model="llama-3.3-70b-versatile",
    )
    return completion_model.choices[0].message.content

## Escrita do System Prompt do agente
Nessa etapa temos o prompt do sistema, que constitui de toda explicação do que e como o loop do agente (ReAct) deve operar

In [40]:
system_prompt = """
Você deve operar em um loop de Thought(Pensamento), Action(Ação), PAUSE, Observation(Observação).
Ao terminar o loop, deve retornar uma resposta.
Utilize Thought para descrever seus pensamentos a cerca da questão que deve responder
Use Action para executar uma das ações avaliadas por você - então retorne PAUSE

As ações disponíveis são:

calculate:
ex.: calculate: 4 * 7 / 3
Executa um cálculo e retorna o número - utiliza Python, portanto, certifique-se de usar a sintaxe de ponto flutuante, se necessário.

get_planet_gravity:
ex.: get_planet_gravity: Marte
retorna a gravidade do planeta em m/s - lembre-se que a unidade é metros por segundo ao quadrado

Exemplo de loop a ser seguido:

Pergunta: Qual o valor da força Peso de uma pessoa em Jupiter, sabendo que ela possui um Peso de 686 newtons na Terra?
Thought: Eu preciso encontrar o valor da gravidade em Jupiter
Action: get_planet_gravity: Jupiter
PAUSE

Você receberá outra chamada com a seguinte mensagem:

Observation: A gravidade em Jupiter é 24.79 m/s^2

Thought: Eu preciso encontrar a massa da pessoa com peso de 686 newtons
Action: calculate: 686 / 9.8
PAUSE

Você receberá outra chamada com a seguinte mensagem:

Observation: A massa da pessoa é 70 kg

Thought: Eu preciso multiplicar 70 kg por 24.79 m/s^2
Action: calculate: 70 * 24.79
PAUSE

Você receberá outra chamada com a seguinte mensagem:

Observation: 1735.3 newtons

Caso tenha encontrado a resposta, mostre como "Resposta"

Resposta: O peso da pessoa é 1735.3 newtons no planeta Jupiter

Agora é sua vez de executar:
""".strip()

## Criação das ferramentas/funções
Nessa etapa temos a criação das ferramentas(tools) que podem ser usadas pelo agente, sendo elas: calculate (para realizar operações matemáticas) e get_planet_gravity (para encontrar os valores das gravidades em cada planeta).

In [31]:
def calculate(operacao):
  return eval(operacao)

def get_planet_gravity(planeta) -> float:
  match planeta.lower():
    case "mercurio":
      return 3.7
    case "venus":
      return 8.87
    case "terra":
      return 9.8
    case "marte":
      return 3.71
    case "saturno":
      return 10.44
    case "jupiter":
      return 24.79
    case "urano":
      return 8.69
    case "netuno":
      return 11.15
    case _:
      return 0.0

## Loop de repetição automática do agente
Nessa etapa automatizamos o processo de execução do agente.

In [41]:
import re

def loop_agent(iterations_max, system, question):
  agent = Agent(client, system_prompt)
  tools_available = ['calculate', 'get_planet_gravity']
  next_prompt = question
  i=0

  while i < iterations_max:
    i+=1
    resultado = agent(next_prompt)
    print(resultado)

    if 'PAUSE' in resultado and 'Action' in resultado:
      action = re.findall(r'Action: ([a-z_]+): (.+)', resultado, re.IGNORECASE)
      choice_tools = action[0][0]
      argument = action[0][1]

      if choice_tools in tools_available:
        result_tool = eval(f"{choice_tools}('{argument}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = f"Observation: Não foi encontrado a tool"

      print(next_prompt)
      continue

    if "Resposta" in resultado:
      break

In [42]:
garoto_gravidade = loop_agent(iterations_max=10, system=system_prompt, question="Qual a força Peso de uma pessoa em Urano, sabendo que ela possui um Peso de 686 newtons na Terra?")

Thought: Eu preciso encontrar o valor da gravidade em Urano e calcular a massa da pessoa com base no seu peso na Terra.
Action: get_planet_gravity: Urano
PAUSE
Observation: 8.69
Thought: Agora que sei a gravidade em Urano, preciso encontrar a massa da pessoa com base no seu peso na Terra, que é 686 newtons, e a gravidade na Terra, que é 9,8 m/s^2.
Action: calculate: 686 / 9.8
PAUSE
Observation: 70.0
Thought: Agora que sei a massa da pessoa, que é 70 kg, posso calcular a força peso em Urano multiplicando a massa pela gravidade em Urano.
Action: calculate: 70 * 8.69
PAUSE
Observation: 608.3
Thought: Eu já tenho todos os dados necessários para calcular a força peso da pessoa em Urano, portanto, posso retornar a resposta.
Resposta: O peso da pessoa é 608.3 newtons no planeta Urano.


Dificuldade: Com o modelo de system_prompt incial que eu utilizei, encontrei certas complicações na hora de sair a resposta, uma vez que ele não estava detalhado o suficiente para que o LLM possa exibir de forma a quebrar o loop assim que encontrar a respota da pergunta